In [4]:
# 1, 3, 5, 6

import pandas as pd
from sklearn.model_selection import StratifiedGroupKFold

strange_series = [
    "05e1944c3818",  # 1
    "13b4d6a01d27",  # 4
    "7476c0bd18d2",  # 3
    "5aad18e7ce64",  # 2
    "aed3850f65f0",  # 5
    "c5365a55ebb7",  # 2
    "f981a0805fd0",  # 4
    "0f9e60a8e56d",  # 3
    "2fc653ca75c7",  # 5
    "390b487231ce",  # 4
    "89c7daa72eee",  # 1
    "a3e59c2ce3f6",  # 5
    "c5d08fc3e040",  # 3
    "c7b1283bb7eb",  # 2
    "e11b9d69f856",  # 1
]

x = pd.read_csv("../data/child-mind-institute-detect-sleep-states/train_events.csv")
x["is_strange"] = 0
x["fold"] = 0
x.loc[x["series_id"].isin(strange_series), "is_strange"] = 1

sgkf = StratifiedGroupKFold(n_splits=4)

x = x[x["event"] == "onset"].reset_index(drop=True)
X = x[["night"]]
y = x["is_strange"]
groups = x["series_id"]

res = ""

for i, (train_index, test_index) in enumerate(sgkf.split(X, y, groups)):
    series = x.loc[test_index, "series_id"].unique()
    x.loc[test_index, "fold"] = i
    print(x.loc[test_index, "step"].isna().sum())

    for s in series:
        res += f"- {s}\n"
    res += "\n"

595
617
538
713


In [5]:
print(res)

- 04f547b8017d
- 0a96f4993bd7
- 0cd1e3d0ed95
- 0d0ad1e77851
- 0f9e60a8e56d
- 12d01911d509
- 1762ab70ec76
- 188d4b7cd28b
- 1c7c0bad1263
- 280e08693c6d
- 292a75c0b94e
- 29c75c018220
- 29d3469bd15d
- 2fbbee1a38e3
- 349c5562ee2c
- 3df0da2e5966
- 4a31811f3558
- 4ab54be1a403
- 4ac356361be9
- 4feda0596965
- 55b7f5c99930
- 599ca4ed791b
- 5c088d7e916c
- 5f76965e10cf
- 5ffd5e1e81ac
- 6bf95a3cf91c
- 6ee4ade1f2bd
- 72bbd1ac3edf
- 7476c0bd18d2
- 76237b9406d5
- 78569a801a38
- 808652a666c6
- 844f54dcab89
- 87a6cbb7c4ed
- 8a306e0890c0
- 9277be28a1cf
- 99b829cbad2d
- 9aed9ee12ae2
- 9ddd40f2cb36
- 9ee455e4770d
- a2b0a64ec9cf
- ad425f3ee76d
- b1831c4979da
- b4b75225b224
- b750c8c1556c
- bb5612895813
- bf00506437aa
- c289c8a823e0
- c38707ef76df
- c5365a55ebb7
- c68260cc9e8f
- c7b1283bb7eb
- c7d693f24684
- cca14d1966c1
- d150801f3145
- d2fef7e4defd
- dacc6d652e35
- dfc3ccebfdc9
- dff367373725
- e1f2a4f991cb
- e30cb792a2bc
- e69aff66e0cb
- e8d0a37c3eba
- ebb6fae8ed43
- ebd76e93ec7d
- f2c2436cf7b7
- f7eb1792

# Ideas

- fix DETR

In [79]:
import json
import numpy as np
import pandas as pd
from scipy.signal import find_peaks
from pathlib import Path
from hydra import initialize, compose
from hydra.core.global_hydra import GlobalHydra
from src.utils.metrics import event_detection_ap

from src.utils.post_process import post_process_for_seg
import jupyter_black
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import optuna


jupyter_black.load()
plt.style.use("ggplot")

INFERENCE = False
EXP_NAME = "transformer_folds_7_lstm"
RUN_NAME = "run0"
TYPE = "score"

if INFERENCE:
    RESULT_DIR = Path("../output/inference") / EXP_NAME  # / RUN_NAME
    hydra_result_dir = Path("../output/train") / "transformer_folds_5" / RUN_NAME
else:
    RESULT_DIR = Path("../output/train") / EXP_NAME
    hydra_result_dir = Path("../output/train") / EXP_NAME / RUN_NAME


def load_config(result_dir: Path):
    # clear previous initialization
    GlobalHydra.instance().clear()

    # initialize hydra
    config_path = result_dir / ".hydra"
    initialize(config_path=config_path.as_posix())
    # load the config
    cfg = compose(config_name="config")

    return cfg


cfg = load_config(hydra_result_dir)

if INFERENCE:
    preds = np.load(RESULT_DIR / f"preds.npy")
    keys = np.load(RESULT_DIR / f"keys.npy")
else:
    preds, labels, keys = None, None, None
    for i in range(7):
        tmp_preds = np.load(RESULT_DIR / f"run{i}" / f"preds_{TYPE}.npy")
        tmp_labels = np.load(RESULT_DIR / f"run{i}" / f"labels_{TYPE}.npy")
        tmp_keys = np.load(RESULT_DIR / f"run{i}" / "keys.npy")

        if tmp_preds.shape[2] == 3:
            tmp_preds = tmp_preds[:, :, [1, 2]]

        if tmp_labels.shape[2] == 3:
            tmp_labels = tmp_labels[:, :, [1, 2]]

        if preds is None:
            preds = tmp_preds
            labels = tmp_labels
            keys = tmp_keys
        else:
            preds = np.concatenate((preds, tmp_preds))
            labels = np.concatenate((labels, tmp_labels))
            keys = np.concatenate((keys, tmp_keys))

gt_df = pd.read_csv(Path(cfg.dir.data_dir) / "train_events.csv").dropna().reset_index(drop=True)
# gt_df = gt_df[gt_df["series_id"].isin(cfg.split.valid_series_ids)]

/tmp/ipykernel_2632110/3043315833.py:41: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path=config_path.as_posix())


# Optimize postprocess parameters using Optuna

In [80]:
def clean_weak_events(sub_df, event_threshold):
    # delete onset and wakeup events that are too far from the same events with high score
    sub_df["prev_onset_step"] = np.nan
    sub_df["next_onset_step"] = np.nan
    sub_df["prev_wakeup_step"] = np.nan
    sub_df["next_wakeup_step"] = np.nan
    sub_df["to_del"] = 0

    # onset
    mask = (sub_df["event"] == "onset") & (sub_df["score"] >= 0.1)
    sub_df.loc[mask, "prev_high_score_onset_step"] = sub_df.loc[mask, "step"]
    sub_df.loc[mask, "next_high_score_onset_step"] = sub_df.loc[mask, "step"]
    sub_df["prev_high_score_onset_step"] = sub_df["prev_high_score_onset_step"].ffill()
    sub_df["next_high_score_onset_step"] = sub_df["next_high_score_onset_step"].bfill()

    mask = (
        (sub_df["event"] == "onset")
        & (sub_df["step"] - sub_df["prev_high_score_onset_step"] >= event_threshold)
        & (sub_df["next_high_score_onset_step"] - sub_df["step"] >= event_threshold)
    )
    sub_df.loc[mask, "to_del"] = 1

    # wakeup
    mask = (sub_df["event"] == "wakeup") & (sub_df["score"] >= 0.1)
    sub_df.loc[mask, "prev_high_score_wakeup_step"] = sub_df.loc[mask, "step"]
    sub_df.loc[mask, "next_high_score_wakeup_step"] = sub_df.loc[mask, "step"]
    sub_df["prev_high_score_wakeup_step"] = sub_df["prev_high_score_wakeup_step"].ffill()
    sub_df["next_high_score_wakeup_step"] = sub_df["next_high_score_wakeup_step"].bfill()

    mask = (
        (sub_df["event"] == "wakeup")
        & (sub_df["step"] - sub_df["prev_high_score_wakeup_step"] >= event_threshold)
        & (sub_df["next_high_score_wakeup_step"] - sub_df["step"] >= event_threshold)
    )
    sub_df.loc[mask, "to_del"] = 1

    # remove these events
    sub_df = sub_df[sub_df["to_del"] == 0]

    return sub_df


def clean_too_far_events(sub_df, event_threshold):
    # delete onset and wakeup events that are too far from next wakeup / previous onset event
    sub_df["prev_onset_step"] = np.nan
    sub_df["next_onset_step"] = np.nan
    sub_df["prev_wakeup_step"] = np.nan
    sub_df["next_wakeup_step"] = np.nan
    sub_df["to_del"] = 0

    # onset
    mask = sub_df["event"] == "onset"
    sub_df.loc[mask, "prev_onset_step"] = sub_df.loc[mask, "step"]
    sub_df.loc[mask, "next_onset_step"] = sub_df.loc[mask, "step"]
    sub_df["prev_onset_step"] = sub_df["prev_onset_step"].ffill()
    sub_df["next_onset_step"] = sub_df["next_onset_step"].bfill()

    mask = sub_df["step"] - sub_df["prev_onset_step"] >= event_threshold
    sub_df.loc[mask, "to_del"] = 1

    # wakeup
    mask = sub_df["event"] == "wakeup"
    sub_df.loc[mask, "prev_wakeup_step"] = sub_df.loc[mask, "step"]
    sub_df.loc[mask, "next_wakeup_step"] = sub_df.loc[mask, "step"]
    sub_df["prev_wakeup_step"] = sub_df["prev_wakeup_step"].ffill()
    sub_df["next_wakeup_step"] = sub_df["next_wakeup_step"].bfill()

    mask = sub_df["next_wakeup_step"] - sub_df["step"] >= event_threshold
    sub_df.loc[mask, "to_del"] = 1

    # remove these events
    sub_df = sub_df[sub_df["to_del"] == 0]

    return sub_df


def delete_alone_events(sub_df, alone_threshold):
    # detect alone onset event that are between two other onset events and delete them
    sub_df["to_del"] = 0

    # onset
    mask_1 = (
        sub_df["step"] - sub_df["prev_onset_step"].shift(1)
        < sub_df["step"] - sub_df["prev_wakeup_step"]
    )
    mask_2 = sub_df["step"] - sub_df["prev_onset_step"].shift(1) >= alone_threshold
    mask_3 = (
        sub_df["next_onset_step"].shift(-1) - sub_df["step"]
        < sub_df["next_wakeup_step"] - sub_df["step"]
    )
    mask_4 = sub_df["next_onset_step"].shift(-1) - sub_df["step"] >= alone_threshold
    mask_5 = sub_df["event"] == "onset"
    mask_6 = sub_df["score"] < 0.1
    sub_df.loc[mask_1 & mask_2 & mask_3 & mask_4 & mask_5 & mask_6, "to_del"] = 1

    # wakeup
    mask_1 = (
        sub_df["step"] - sub_df["prev_wakeup_step"].shift(1)
        < sub_df["step"] - sub_df["prev_onset_step"]
    )
    mask_2 = sub_df["step"] - sub_df["prev_wakeup_step"].shift(1) >= alone_threshold
    mask_3 = (
        sub_df["next_wakeup_step"].shift(-1) - sub_df["step"]
        < sub_df["next_onset_step"] - sub_df["step"]
    )
    mask_4 = sub_df["next_wakeup_step"].shift(-1) - sub_df["step"] >= alone_threshold
    mask_5 = sub_df["event"] == "wakeup"
    mask_6 = sub_df["score"] < 0.1
    sub_df.loc[mask_1 & mask_2 & mask_3 & mask_4 & mask_5 & mask_6, "to_del"] = 1

    # remove these events
    sub_df = sub_df[sub_df["to_del"] == 0]

    return sub_df


def delete_events_among_differnet_evnets(sub_df, close_threshold):
    # delete events that are among many different events
    sub_df["to_del"] = 0

    # onset
    mask_1 = (
        (sub_df["event"].shift(1) == "wakeup")
        & (sub_df["event"].shift(2) == "wakeup")
        & (sub_df["event"].shift(3) == "wakeup")
    )
    mask_2 = (
        (sub_df["event"].shift(-1) == "wakeup")
        & (sub_df["event"].shift(-2) == "wakeup")
        & (sub_df["event"].shift(-3) == "wakeup")
    )
    mask_3 = sub_df["event"] == "onset"
    mask_4 = sub_df["step"] - sub_df["prev_wakeup_step"] <= close_threshold
    mask_5 = sub_df["next_wakeup_step"] - sub_df["step"] <= close_threshold
    mask_6 = sub_df["score"] < 0.1

    sub_df.loc[mask_1 & mask_2 & mask_3 & mask_4 & mask_5 & mask_6, "to_del"] = 1

    # wakeup
    mask_1 = (
        (sub_df["event"].shift(1) == "onset")
        & (sub_df["event"].shift(2) == "onset")
        & (sub_df["event"].shift(3) == "onset")
    )
    mask_2 = (
        (sub_df["event"].shift(-1) == "onset")
        & (sub_df["event"].shift(-2) == "onset")
        & (sub_df["event"].shift(-3) == "onset")
    )
    mask_3 = sub_df["event"] == "wakeup"
    mask_4 = sub_df["step"] - sub_df["prev_onset_step"] <= 360
    mask_5 = sub_df["next_onset_step"] - sub_df["step"] <= 360
    mask_6 = sub_df["score"] < 0.1
    sub_df.loc[mask_1 & mask_2 & mask_3 & mask_4 & mask_5 & mask_6, "to_del"] = 1

    # remove these events
    sub_df = sub_df[sub_df["to_del"] == 0]

    return sub_df


def post_process_for_seg(
    keys: list[str],
    preds: np.ndarray,
    series_lens: dict,
    score_th: float = 0.01,
    distance: int = 5000,
    offset: int = 0,
) -> pd.DataFrame:
    """make submission dataframe for segmentation task

    Args:
        keys (list[str]): list of keys. key is "{series_id}_{chunk_id}"
        preds (np.ndarray): (num_series * num_chunks, duration, 2)
        score_th (float, optional): threshold for score. Defaults to 0.5.

    Returns:
        pd.DataFrame: submission dataframe
    """
    series_ids = np.array(list(map(lambda x: x.split("_")[0], keys)))
    unique_series_ids = np.unique(series_ids)

    records = []
    for series_id in unique_series_ids:
        max_step = series_lens[series_id]

        series_idx = np.where(series_ids == series_id)[0]
        this_series_preds = preds[series_idx].reshape(-1, 2)

        onset_event_preds = this_series_preds[:, 0]
        onset_steps = find_peaks(onset_event_preds, height=score_th, distance=distance)[0]
        onset_scores = onset_event_preds[onset_steps]
        min_onset_step = min(onset_steps) if len(onset_steps) > 0 else 0

        wakeup_event_preds = this_series_preds[:, 1]
        wakeup_steps = find_peaks(wakeup_event_preds, height=score_th, distance=distance)[0]
        wakeup_scores = wakeup_event_preds[wakeup_steps]
        max_wakeup_step = max(wakeup_steps) if len(wakeup_steps) > 0 else 0

        for step, score in zip(onset_steps, onset_scores):
            # select only wakeups than has at least one onset before
            # and not too close to series borders
            if step >= max_wakeup_step or step <= 720 or step >= max_step - 720 * offset:
                continue

            records.append(
                {
                    "series_id": series_id,
                    "step": step,
                    "event": "onset",
                    "score": score,
                }
            )

        for step, score in zip(wakeup_steps, wakeup_scores):
            # select only onsets than has at least one wakeup after
            # and not too close to series borders
            if step <= min_onset_step or step <= 720 * offset or step >= max_step - 180:
                continue

            records.append(
                {
                    "series_id": series_id,
                    "step": step,
                    "event": "wakeup",
                    "score": score,
                }
            )

    if len(records) == 0:  # If there is no prediction, insert dummy
        records.append(
            {
                "series_id": series_id,
                "step": 0,
                "event": "onset",
                "score": 0,
            }
        )

    sub_df = pd.DataFrame(records).sort_values(["series_id", "step"]).reset_index(drop=True)
    sub_df["row_id"] = np.arange(len(sub_df))

    # # delete onset and wakeup events that are too far from the same events with high score
    # high_score_event_threshold = 17280 * 1.01  # 24 hours
    # sub_df = clean_weak_events(sub_df, high_score_event_threshold)

    # # delete onset and wakeup events that are too far from next wakeup / previous onset event
    # event_threshold = 14400  # 20 hours
    # sub_df = clean_too_far_events(sub_df, event_threshold)

    # # detect alone onset event that are between two other onset events and delete them
    # alone_threshold = 2880  # 2 hours
    # sub_df = delete_alone_events(sub_df, alone_threshold)

    # # delete events that are among many different events
    # close_threshold = 360  # 30 minutes
    # sub_df = delete_events_among_differnet_evnets(sub_df, close_threshold)

    sub_df = sub_df[["row_id", "series_id", "step", "event", "score"]]

    return sub_df


with open(Path(cfg.dir.processed_dir) / "train" / "series_lens.json") as f:
    series_lens = json.load(f)

pred_df = post_process_for_seg(keys, preds, series_lens, score_th=0.0001, distance=70, offset=8)
# pred_df = pred_df.to_pandas()

score = event_detection_ap(gt_df, pred_df)

score, len(pred_df)

(0.6707851650592838, 470660)

In [18]:
# (0.7675050274210242, 468368)

In [ ]:
# def objective(trial: optuna.Trial):
#     # score_th = 0.005
#     score_th = trial.suggest_float("score_th", 0, 0.5)
#     distance = trial.suggest_int("distance", 30, 400)

#     pred_df: pl.DataFrame = post_process_for_seg(keys, preds, score_th=score_th, distance=distance)
#     score = event_detection_ap(gt_df, pred_df)

#     return score

# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=100)

# print(study.best_params)

In [ ]:
def plot_random_sample(gt_df, keys, preds, labels, num_samples=1, num_chunks=10):
    # get series ids
    series_ids = np.array(list(map(lambda x: x.split("_")[0], keys)))
    unique_series_ids = np.unique(series_ids)

    # get random series
    random_series_ids = np.random.choice(unique_series_ids, num_samples)

    for i, random_series_id in enumerate(random_series_ids):
        # get random series
        series_idx = np.where(series_ids == random_series_id)[0]
        this_series_preds = preds[series_idx].reshape(-1, 3)
        this_series_labels = labels[series_idx].reshape(-1, 3)

        # split series
        this_series_preds = np.split(this_series_preds, num_chunks)
        this_series_labels = np.split(this_series_labels, num_chunks)
        this_series_len = [0] + [len(x) for x in this_series_labels]
        this_series_len = np.cumsum(this_series_len)

        gt_df = gt_df[gt_df["series_id"] == random_series_id]

        fig, axs = plt.subplots(num_chunks, 1, figsize=(20, 5 * num_chunks))

        if num_chunks == 1:
            axs = [axs]

        for j in range(num_chunks):
            this_series_preds_chunk = this_series_preds[j]
            this_series_labels_chunk = this_series_labels[j]

            # get onset and wakeup idx
            gt_tmp = gt_df[
                (gt_df["step"] >= this_series_len[j]) & (gt_df["step"] <= this_series_len[j + 1])
            ]
            onset_idx = gt_tmp.loc[gt_tmp["event"] == "onset", "step"].to_list()
            onset_idx = onset_idx - this_series_len[j]
            wakeup_idx = gt_tmp.loc[gt_tmp["event"] == "wakeup", "step"].to_list()
            wakeup_idx = wakeup_idx - this_series_len[j]

            axs[j].plot(this_series_preds_chunk[:, 0], label="pred_sleep")
            axs[j].plot(this_series_preds_chunk[:, 1], label="pred_onset")
            axs[j].plot(this_series_preds_chunk[:, 2], label="pred_wakeup")
            axs[j].vlines(onset_idx, 0, 1, label="onset", linestyles="dashed", color="C1")
            axs[j].vlines(wakeup_idx, 0, 1, label="wakeup", linestyles="dashed", color="C2")
            axs[j].set_ylim(0, 1)
            axs[j].set_title(f"series_id: {random_series_id} chunk_id: {j}")
            axs[j].legend(bbox_to_anchor=(1.05, 1), loc="upper left", borderaxespad=0)

    plt.tight_layout()


plot_random_sample(gt_df, keys, preds, labels, num_chunks=10)

# Inference

In [1]:
# Config
BEST_MODEL = "ensemble"
FOLD = 0

DURATION = 8640
DOWNSAMPLE_RATE = 2
PHASE = "train"
EXP_NAME = "transformer_best_folds"

In [2]:
!python -m run.inference\
    dir=local\
    model.params.encoder_name=resnet34\
    model.params.encoder_weights=null\
    num_workers=12\
    exp_name=$EXP_NAME\
    weight.run_name=single\
    batch_size=8\
    duration=$DURATION\
    downsample_rate=$DOWNSAMPLE_RATE\
    pp.score_th=0.0015\
    pp.distance=70\
    phase=$PHASE\
    best_model=$BEST_MODEL

Global seed set to 42
[5.0GB(+3.9GB):2.2sec] load test dataloader 
List of models: ['/home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_0.pth', '/home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_10_2.pth', '/home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_10_4.pth', '/home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_3.pth']
load weight from /home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_0.pth
load weight from /home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_10_2.pth
load weight from /home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_10_4.pth
load weight from /home/alex/Kaggle/DSS/output/train/transformer_best_folds/fold_3.pth
[5.7GB(+0.7GB):4.8sec] load model 
inference: 100%|████████████████████████████| 1867/1867 [02:38<00:00, 11.80it/s]
[7.4GB(+1.7GB):177.8sec] inference 
inference: 100%|████████████████████████████| 1867/1867 [02:38<00:00, 11.77it/s]
[7.4GB(+0.0GB):180.0sec] inference 
inf